In [ ]:
from abc import ABC, abstractmethod

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from sklearn.linear_model import LogisticRegression
from statsmodels.discrete.discrete_model import Probit

stimulus_range = (9, 16)
num_trials = 10000
bias = 0
stim_weight = 4


In [ ]:
class Model(ABC):
    def __init__(self, bias: float, stim_weight: float):
        self._bias = bias
        self._stim_weight = stim_weight

    @abstractmethod
    def __call__(self, stimuli: np.ndarray) -> np.ndarray:
        raise NotImplementedError


class LogisticModel(Model):
    def __call__(self, stimuli: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-(self._bias + self._stim_weight * stimuli)))


class ProbitModel(Model):
    def __call__(self, stimuli: np.ndarray) -> np.ndarray:
        return norm.cdf(self._bias + self._stim_weight * stimuli)


def simulate_data(
    stimulus_range: tuple[int, int], num_trials: int, model: Model
) -> tuple[np.ndarray, np.ndarray]:
    """
    Simulates data from perceptual decision-making experiment using logistic regression.

    Args:
        stimulus_range: Range of values stimuli can take on.
        num_trials: Number of trials to simulate.

    Returns:
        stimuli: Stimuli presented to the subject.
        responses: Subject's responses.
    """
    stimuli = np.random.uniform(*stimulus_range, size=num_trials)
    threshold = np.mean(stimulus_range)
    stimuli = stimuli - threshold
    probabilities = model(stimuli)
    responses = np.random.binomial(1, probabilities)
    return stimuli, responses



In [ ]:
model = LogisticModel(bias, stim_weight)
stimuli, responses = simulate_data(stimulus_range, num_trials, model)
sorted_stimuli = np.sort(stimuli)


In [ ]:
lr = LogisticRegression()
lr.fit(stimuli.reshape(-1, 1), responses)
probabilities = lr.predict_proba(sorted_stimuli.reshape(-1, 1))[:, 1]


In [ ]:
plt.scatter(stimuli, responses, label="responses")
true_y = model(sorted_stimuli)
fit_y = probabilities
plt.plot(sorted_stimuli, true_y, color="red", label="true model")
plt.plot(sorted_stimuli, fit_y, color="green", label="fit model")
plt.legend()


In [ ]:
model = ProbitModel(bias, stim_weight)
stimuli, responses = simulate_data(stimulus_range, num_trials, model)
sorted_stimuli = np.sort(stimuli)


In [ ]:
probit = Probit(responses, stimuli)
probit_results = probit.fit()
probabilities = probit_results.predict(sorted_stimuli)


In [ ]:
plt.scatter(stimuli, responses, label="responses")
true_y = model(sorted_stimuli)
fit_y = probabilities
plt.plot(sorted_stimuli, true_y, color="red", label="true model")
plt.plot(sorted_stimuli, fit_y, color="green", label="fit model")
plt.legend()
